# **Assignment 4: PySpark Structured Streaming Using Kafka Source**

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-kafka-streaming").\
        master("spark://spark-master:7077").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0"). \
        config("spark.executor.memory", "512m").\
        getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e6bd46ec-7ead-4f56-8986-e6e3ea12c7ad;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 997ms :: artifacts dl 25

## ==== Q2 ====

#### **Q2.1:** All your code for 2.1 should be in the following cell

In [2]:
#Answer to 2.1
df_streamed_raw = (spark
  .readStream
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9093") \
  .option("subscribe", "topic_test1") \
  .load())


In [3]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import col

# convert byte stream to string
df_streamed_kv = (df_streamed_raw
    .withColumn("key", df_streamed_raw["key"].cast(StringType()))
    .withColumn("value", df_streamed_raw["value"].cast(StringType())))

test_query = (df_streamed_kv 
              .writeStream \
              .format("memory") # output to memory \
              .outputMode("update") # only write updated rows to the sink \
              .queryName("test_query_table")  # Name of the in memory table \
              .start())

23/04/12 16:25:59 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-66229bb6-00a2-4e8e-8f04-969f38ef27ec. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


#### If all goes well, the following cell should display a table populated with values being streamed from you Kafka producer. NOTE: If you recently ran the producer, it may take a while before the table is populated. Keep rerunning the cell to check for updates:

In [ ]:
spark.sql("select * from test_query_table").show()

In [ ]:
test_query.stop()

#### The following cells contain code that take the streamed dataframe and formats it properly into a table. If any of the given cells fails, there might be a formatting issue with one of your previous solutions. 

In [4]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, BooleanType, LongType, IntegerType

event_schema = StructType([
    StructField("station", StringType()),
    StructField("valid", StringType()),
    StructField("tmpf", StringType()),
    StructField("dwpf", StringType()),
    StructField("relh", StringType()),
    StructField("feel", StringType()),
    StructField("drct", StringType()),
    StructField("sped", StringType()),
    StructField("alti", StringType()),
    StructField("mslp", StringType()),
    StructField("p01m", StringType()),
    StructField("vsby", StringType()),
    StructField("skyc1", StringType()),
    StructField("skyl1", StringType()),
    StructField("wxcodes", StringType()),
    StructField("ice_acceretion_1hr", StringType()),
])

# Parse the events from JSON format
df_parsed = (df_streamed_kv
           # Sets schema for event data
           .withColumn("value", from_json("value", event_schema))
          )

In [5]:
from pyspark.sql.functions import to_timestamp, unix_timestamp

# Here, we need to convert date_time string to date_time object in the "dd/MMM/yyyy:HH:mm:ss Z" format.

df_formatted = (df_parsed.select(
    col("key").alias("event_key")
    ,col("topic").alias("event_topic")
    ,col("timestamp").alias("event_timestamp")
    ,col("value.station").alias("station")
#     ,col("value.valid").alias("valid")
    ,to_timestamp(col("value.valid"), "yyyy-MM-dd HH:mm").alias("valid")
    ,col("value.tmpf").alias("tmpf")
    ,col("value.dwpf").alias("dwpf")
    ,col("value.relh").alias("relh")
    ,col("value.feel").alias("feel")
    ,col("value.drct").alias("drct")
    ,col("value.sped").alias("sped")
    ,col("value.alti").alias("alti")
    ,col("value.mslp").alias("mslp")
    ,col("value.p01m").alias("p01m")
    ,col("value.vsby").alias("vsby")
    ,col("value.skyc1").alias("skyc1")
    ,col("value.skyl1").alias("skyl1")
    ,col("value.wxcodes").alias("wxcodes")
    ,col("value.ice_acceretion_1hr").alias("ice_acceretion_1hr")
#     cast(IntegerType()).
))

#### **Q2.2:** All your code for 2.2 should be in the following cell


In [6]:
# Answer to 2.2
query = (df_formatted
            .writeStream
            .format("console")
            .trigger(processingTime='5 seconds')
            .outputMode("append")
            .option("truncate",'false')
            .start()
        )

23/04/12 16:26:02 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7f524734-f72b-49e3-aabe-804bc14480e8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------


+---------+-----------+---------------+-------+-----+----+----+----+----+----+----+----+----+----+----+-----+-----+-------+------------------+
|event_key|event_topic|event_timestamp|station|valid|tmpf|dwpf|relh|feel|drct|sped|alti|mslp|p01m|vsby|skyc1|skyl1|wxcodes|ice_acceretion_1hr|
+---------+-----------+---------------+-------+-----+----+----+----+----+----+----+----+----+----+----+-----+-----+-------+------------------+
+---------+-----------+---------------+-------+-----+----+----+----+----+----+----+----+----+----+----+-----+-----+-------+------------------+



23/04/12 16:26:08 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6190 milliseconds


In [ ]:
# Print the name of active streams (This may be useful during debugging)
for s in spark.streams.active:
    print(f"ID:{s.id} | NAME:{s.name}")

In [7]:
query.stop()

23/04/12 16:26:15 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@552debf2 is aborting.
23/04/12 16:26:15 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@552debf2 aborted.
23/04/12 16:26:18 WARN TaskSetManager: Lost task 0.0 in stage 2.0 (TID 2, 172.18.0.6, executor 1): TaskKilled (Stage cancelled)


# ==== Project - Start your feature extraction queries from here ====

## Weekly

In [ ]:
query_4 = df_stream_night\
    .writeStream\
    .format("console")\
    .trigger(processingTime="10 seconds")\
    .outputMode("complete")\
    .option("truncate", "false")\
    .start()

In [ ]:
query_4.stop()

In [ ]:
# df_day_night=df_stream_day.union(df_stream_night)

In [ ]:
# query_day_night = df_day_night\
#     .writeStream\
#     .format("console")\
#     .trigger(processingTime="10 seconds")\
#     .outputMode("append")\
#     .option("truncate", "false")\
#     .start()

In [ ]:
# query_day_night.stop()

In [8]:
from pyspark.sql.functions import avg, col, window,stddev
# assuming your input stream is named `input_stream`
df_stream = (df_formatted
    .groupBy("station", window("valid", "7 days", "1 day"))\
    .agg(avg("tmpf").alias("avg_temp"), stddev("tmpf").alias("stddev_temp")))


In [9]:
#batch

first_aggregation_query = df_stream.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("first_aggregation") \
    .trigger(processingTime="10 seconds") \
    .start()

23/04/12 16:26:39 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cece2d47-7ade-43db-b36b-239cbb2d6148. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [19]:
#batch
first_aggregation_query.stop()

23/04/12 16:29:50 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@64744448 is aborting.
23/04/12 16:29:50 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@64744448 aborted.
23/04/12 16:29:50 WARN TaskSetManager: Lost task 39.0 in stage 61.0 (TID 2484, 172.18.0.6, executor 1): TaskKilled (Stage cancelled)
23/04/12 16:29:50 WARN TaskSetManager: Lost task 38.0 in stage 61.0 (TID 2483, 172.18.0.7, executor 0): TaskKilled (Stage cancelled)


In [48]:
#batch

first_aggregation_output = spark.sql("SELECT * FROM first_aggregation")

-------------------------------------------
Batch: 103
-------------------------------------------


+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|Window                                    |station|temperature|station|window                                    |avg_temp|stddev_temp|
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2013-01-01 00:00:00, 2013-01-08 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-28 00:00:00, 2013-01-04 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-31 00:00:00, 2013-01-07 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:0

23/04/12 17:08:56 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 30712 milliseconds


-------------------------------------------
Batch: 133
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2

23/04/12 17:09:09 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 35025 milliseconds
23/04/12 17:09:17 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 36520 milliseconds


...

23/04/12 17:09:51 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 34309 milliseconds
23/04/12 17:10:04 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 32387 milliseconds


-------------------------------------------
Batch: 135
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2

-------------------------------------------
Batch: 106
-------------------------------------------
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|Window                                    |station|temperature|station|window                                    |avg_temp|stddev_temp|
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2013-01-01 00:00:00, 2013-01-08 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-28 00:00:00, 2013-01-04 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-1

23/04/12 17:10:13 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 33379 milliseconds
23/04/12 17:10:22 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 30656 milliseconds


-------------------------------------------
Batch: 72
-------------------------------------------
+------+-------+-----------+-------+------+--------+-----------+
|Window|station|temperature|station|window|avg_temp|stddev_temp|
+------+-------+-----------+-------+------+--------+-----------+
+------+-------+-----------+-------+------+--------+-----------+



In [ ]:
query_stream = df_stream\
    .writeStream\
    .format("console")\
    .trigger(processingTime="10 seconds")\
    .outputMode("update")\
    .option("truncate", "false")\
    .start()

In [ ]:
query_stream.stop()

## 2-hour Window 

In [14]:
from pyspark.sql.functions import *

from pyspark.sql.functions import window
df_windowed_2hrs = (df_formatted
#     .withWatermark("valid", "30 minutes")
    .groupBy("station", window("valid", "2 hours","1 hour"))\
    .agg(collect_list("tmpf").alias("temp_list")))

23/04/12 16:29:04 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 11213 milliseconds
23/04/12 16:29:15 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 10351 milliseconds


In [15]:
query_5=df_windowed_2hrs\
    .writeStream\
    .format("console")\
    .trigger(processingTime="10 seconds")\
    .outputMode("complete")\
    .option("truncate", "false")\
    .start()

23/04/12 16:29:18 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-71ff4c1b-4471-4dde-9d06-3821fb46d505. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [16]:
query_5.stop()

23/04/12 16:29:23 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@33ac857b is aborting.
23/04/12 16:29:23 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@33ac857b aborted.


In [17]:
from pyspark.sql.functions import explode

df_exploded = df_windowed_2hrs.select("Window", "station", explode("temp_list").alias("temperature"))

23/04/12 16:29:26 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 11546 milliseconds


In [35]:
df_exploded = df_exploded.withColumn("prev_week_start", date_sub("window.start", 7))
df_exploded = df_exploded.withColumn("prev_week_end", date_sub("window.start", 1))

In [36]:
query_6= (df_exploded
            .writeStream
            .outputMode("complete")
            .format("console")
            .option("truncate",'false')
            .trigger(processingTime="10 seconds")
            .start())

23/04/12 16:54:06 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f132deeb-e011-48c9-a891-c88325108b4a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/12 16:54:08 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 17296 milliseconds


-------------------------------------------
Batch: 35
-------------------------------------------
+------+-------+-----------+-------+------+--------+-----------+
|Window|station|temperature|station|window|avg_temp|stddev_temp|
+------+-------+-----------+-------+------+--------+-----------+
+------+-------+-----------+-------+------+--------+-----------+



-------------------------------------------
Batch: 69
-------------------------------------------
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|Window                                    |station|temperature|station|window                                    |avg_temp|stddev_temp|
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |VSF    |[2012-12-30 00:00:00, 2013-01-06 00:00:00]|32.0    |0.0        |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |VSF    |[2012-12-28 00:00:00, 2013-01-04 00:00:00]|32.0    |0.0        |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |VSF    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|32.0    |0.0        |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |VSF    |[2013-01

23/04/12 16:54:15 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 34262 milliseconds


-------------------------------------------
Batch: 96
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 18:00:00, 2013-01-01 20:00:00]|EFK    |12.02      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|MVL    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|MVL    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|MVL    |32.0 

23/04/12 16:54:23 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 22779 milliseconds
23/04/12 16:54:29 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 22487 milliseconds


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----------+---------------+-------------+
|Window|station|temperature|prev_week_start|prev_week_end|
+------+-------+-----------+---------------+-------------+
+------+-------+-----------+---------------+-------------+



-------------------------------------------
Batch: 97
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 18:00:00, 2013-01-01 20:00:00]|EFK    |12.02      |
|[2013-01-01 18:00:00, 2013-01-01 20:00:00]|EFK    |12.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|MVL    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|MVL    |30.2 

23/04/12 16:54:36 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 13157 milliseconds


In [45]:
query_6.stop()

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+--------+-------------------+-------------------+
|Window                                    |station|temperature|prev_week_start|prev_week_end|station|window                                    |avg_temp|stddev_temp        |z_score            |
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+--------+-------------------+-------------------+
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |13.64      |2012-12-25     |2012-12-31   |CDA    |[2012-12-26 00:00:00, 2013-01-02 00:00:00]|23.252  |0.5489262245511686 |-17.510549815431542|
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |13.64      |2012-12-25     |2012-12-31   |CDA    |[2012-12-31 00:00:00,

23/04/12 16:58:18 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 45014 milliseconds


-------------------------------------------
Batch: 77
-------------------------------------------


23/04/12 16:58:25 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 36103 milliseconds


+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|Window                                    |station|temperature|station|window                                    |avg_temp|stddev_temp|
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |VSF    |[2012-12-30 00:00:00, 2013-01-06 00:00:00]|32.0    |0.0        |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |VSF    |[2012-12-28 00:00:00, 2013-01-04 00:00:00]|32.0    |0.0        |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |VSF    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|32.0    |0.0        |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |VSF    |[2013-01-01 00:00:00, 2013-01-08 00:00:00]|32.0    |0.0        |
|[2013-01-01 14:00:00, 2013-01-01 16:00:0

...

23/04/12 16:58:31 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 27466 milliseconds


-------------------------------------------
Batch: 106
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.0       |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.36      |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |13.64      |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |12.9

23/04/12 16:58:37 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 27020 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+--------+-------------------+-------------------+
|Window                                    |station|temperature|prev_week_start|prev_week_end|station|window                                    |avg_temp|stddev_temp        |z_score            |
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+--------+-------------------+-------------------+
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |13.64      |2012-12-25     |2012-12-31   |CDA    |[2012-12-26 00:00:00, 2013-01-02 00:00:00]|23.252  |0.5489262245511686 |-17.510549815431542|
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |13.64      |2012-12-25     |2012-12-31   |CDA    |[2012-12-31 00:00:00,

23/04/12 16:58:44 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 26217 milliseconds


### Join

In [49]:
from pyspark.sql import functions as F

comparison_df = df_exploded.join(
    first_aggregation_output,
    (df_exploded["station"] == first_aggregation_output["station"]) 
       & (first_aggregation_output["window.start"] >= df_exploded["prev_week_start"])
    & (first_aggregation_output["window.start"] <= df_exploded["prev_week_end"])
)

In [50]:
#batch

comparison_query = comparison_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="10 seconds") \
    .start()

23/04/12 17:10:32 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7ed384e1-ddd3-45d9-8436-6745e812387e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 136
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2

23/04/12 17:10:33 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 29369 milliseconds


-------------------------------------------
Batch: 107
-------------------------------------------
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|Window                                    |station|temperature|station|window                                    |avg_temp|stddev_temp|
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2013-01-01 00:00:00, 2013-01-08 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-28 00:00:00, 2013-01-04 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-1

23/04/12 17:10:42 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 29642 milliseconds


...

-------------------------------------------
Batch: 137
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2

23/04/12 17:11:07 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 33513 milliseconds
23/04/12 17:11:19 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 46335 milliseconds


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----------+---------------+-------------+-------+------+--------+-----------+
|Window|station|temperature|prev_week_start|prev_week_end|station|window|avg_temp|stddev_temp|
+------+-------+-----------+---------------+-------------+-------+------+--------+-----------+
+------+-------+-----------+---------------+-------------+-------+------+--------+-----------+



-------------------------------------------
Batch: 108
-------------------------------------------
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|Window                                    |station|temperature|station|window                                    |avg_temp|stddev_temp|
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2013-01-01 00:00:00, 2013-01-08 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-28 00:00:00, 2013-01-04 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-1

23/04/12 17:11:27 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 44693 milliseconds


In [44]:
comparison_query.stop()

-------------------------------------------
Batch: 105
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.0       |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.36      |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |13.64      |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |12.9

23/04/12 16:58:10 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 31019 milliseconds


### z_score

In [51]:
df_with_zscore = comparison_df.withColumn('z_score', expr('(temperature - avg_temp) / stddev_temp'))

23/04/12 17:11:35 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 37946 milliseconds


-------------------------------------------
Batch: 74
-------------------------------------------
+------+-------+-----------+-------+------+--------+-----------+
|Window|station|temperature|station|window|avg_temp|stddev_temp|
+------+-------+-----------+-------+------+--------+-----------+
+------+-------+-----------+-------+------+--------+-----------+



-------------------------------------------
Batch: 138
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2

23/04/12 17:11:44 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 37168 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+--------+------------------+
|Window                                    |station|temperature|prev_week_start|prev_week_end|station|window                                    |avg_temp|stddev_temp       |
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+--------+------------------+
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |2012-12-26     |2013-01-01   |FSO    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|31.055  |1.0936635680134912|
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |2012-12-26     |2013-01-01   |FSO    |[2012-12-30 00:00:00, 2013-01-06 00:00:00]|31.055  |1.0936635680134912|
|[2013-01-02 05:00:00, 2013-01-02

...

+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|Window                                    |station|temperature|station|window                                    |avg_temp|stddev_temp|
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2013-01-01 00:00:00, 2013-01-08 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-28 00:00:00, 2013-01-04 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-31 00:00:00, 2013-01-07 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:0

23/04/12 17:23:19 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 335819 milliseconds


-------------------------------------------
Batch: 147
-------------------------------------------


+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.46       |
|[2013-01-02 05:00:00, 2013-01-02 07:00:00]|FSO    |4.82       |
|[2013-01-02 05:00:00, 20

23/04/12 17:24:20 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 275898 milliseconds


In [42]:
df_anomalous_requests = (df_with_zscore.filter(~isnan(col("z_score")))
                      .filter("z_score > 2 or z_score < -2"))

-------------------------------------------
Batch: 102
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.0       |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.36      |
|[2013-01-01 18:00:00, 2013-01-01 20:00:00]|EFK    |12.02      |
|[2013-01-01 18:00:00, 2013-01-01 20:00:00]|EFK    |12.2

23/04/12 16:56:36 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 21160 milliseconds


In [43]:
final_query= (df_anomalous_requests
            .writeStream
            .outputMode("complete")
            .format("console")
            .option("truncate",'false')
            .trigger(processingTime="10 seconds")
            .start())

23/04/12 16:56:37 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6b13bbb8-49f4-46fe-8e90-c7108d82b876. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+------------------+-------------------+
|Window                                    |station|temperature|prev_week_start|prev_week_end|station|window                                    |avg_temp          |stddev_temp        |
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+------------------+-------------------+
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.0       |2012-12-25     |2012-12-31   |CDA    |[2012-12-26 00:00:00, 2013-01-02 00:00:00]|23.252            |0.5489262245511686 |
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.0       |2012-12-25     |2012-12-31   |CDA    |[2012-12-31 00:00:00, 2013-01-07 00:00:00]|23.252            

23/04/12 16:56:44 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 35474 milliseconds


-------------------------------------------
Batch: 74
-------------------------------------------


23/04/12 16:56:53 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 30391 milliseconds


...

23/04/12 16:57:49 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 37547 milliseconds


-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+--------+------------------+
|Window                                    |station|temperature|prev_week_start|prev_week_end|station|window                                    |avg_temp|stddev_temp       |
+------------------------------------------+-------+-----------+---------------+-------------+-------+------------------------------------------+--------+------------------+
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.0       |2012-12-25     |2012-12-31   |CDA    |[2012-12-26 00:00:00, 2013-01-02 00:00:00]|23.252  |0.5489262245511686|
|[2013-01-01 19:00:00, 2013-01-01 21:00:00]|CDA    |14.0       |2012-12-25     |2012-12-31   |CDA    |[2012-12-31 00:00:00, 2013-01-07 00:00:00]|23.252  |0.5489262245511686|
|[2013-01-01 19:00:00, 2013-01-01

23/04/12 16:57:56 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 38087 milliseconds


-------------------------------------------
Batch: 42
-------------------------------------------
+------+-------+-----------+-------+------+--------+-----------+
|Window|station|temperature|station|window|avg_temp|stddev_temp|
+------+-------+-----------+-------+------+--------+-----------+
+------+-------+-----------+-------+------+--------+-----------+



23/04/12 16:58:03 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 38957 milliseconds


In [47]:
final_query.stop()

-------------------------------------------
Batch: 113
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|FSO    |6.98       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|FSO    |6.8 

23/04/12 17:01:17 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 25255 milliseconds


-------------------------------------------
Batch: 51
-------------------------------------------
+------+-------+-----------+-------+------+--------+-----------+
|Window|station|temperature|station|window|avg_temp|stddev_temp|
+------+-------+-----------+-------+------+--------+-----------+
+------+-------+-----------+-------+------+--------+-----------+



23/04/12 17:01:22 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 19095 milliseconds


-------------------------------------------
Batch: 84
-------------------------------------------
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|Window                                    |station|temperature|station|window                                    |avg_temp|stddev_temp|
+------------------------------------------+-------+-----------+-------+------------------------------------------+--------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-29 00:00:00, 2013-01-05 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2013-01-01 00:00:00, 2013-01-08 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12-28 00:00:00, 2013-01-04 00:00:00]|30.2    |0.0        |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |RUT    |[2012-12

...

23/04/12 17:08:26 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 43151 milliseconds


-------------------------------------------
Batch: 132
-------------------------------------------
+------------------------------------------+-------+-----------+
|Window                                    |station|temperature|
+------------------------------------------+-------+-----------+
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |14.0       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 22:00:00, 2013-01-02 00:00:00]|RUT    |12.2       |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |26.06      |
|[2013-01-01 14:00:00, 2013-01-01 16:00:00]|VSF    |24.98      |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2       |
|[2013-01-01 06:00:00, 2013-01-01 08:00:00]|RUT    |30.2

23/04/12 17:08:34 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 27571 milliseconds


-------------------------------------------
Batch: 69
-------------------------------------------
+------+-------+-----------+-------+------+--------+-----------+
|Window|station|temperature|station|window|avg_temp|stddev_temp|
+------+-------+-----------+-------+------+--------+-----------+
+------+-------+-----------+-------+------+--------+-----------+



23/04/12 17:08:41 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 24578 milliseconds
